In [279]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Imports

In [280]:
import pandas as pd

from src.config import SHEET_NAME_PATTERN
from src.course_stage_semester_specialty import (
    create_course_id,
    get_course_stage_semester_specialty,
)
from src.group import create_groups_dim
from src.reading import drop_empty_days, read_and_combine_sheets
from src.rooms import create_room_dim, preprocess_rooms
from src.subject import create_subjects_dim
from src.teachers import create_teachers_dim, get_all_teachers

### Read All sheets

In [281]:
hours_and_teachers_combined = read_and_combine_sheets(
    "./data/raw/Hours and Teachers.xlsx"
)
timetables_combined = read_and_combine_sheets("./data/raw/Timetables.xlsx")
timetables_combined = drop_empty_days(timetables_combined)

### Course, Stage, Semester

In [282]:
sheet_names = hours_and_teachers_combined.sheet_name.unique()
courses_stages_semesters = pd.DataFrame(
    [get_course_stage_semester_specialty(sheet_name) for sheet_name in sheet_names],
    columns=SHEET_NAME_PATTERN,
)


courses = courses_stages_semesters[["course_name", "specialty"]].drop_duplicates()
courses = courses.set_index(create_course_id(courses))
stages = courses_stages_semesters[["stage_name"]].drop_duplicates()
semesters = courses_stages_semesters[["semester_num"]].drop_duplicates()

courses.to_excel("data/dim/courses_dim.xlsx", index=False)
stages.to_excel("data/dim/stages_dim.xlsx", index=False)
semesters.to_excel("data/dim/semesters_dim.xlsx", index=False)

### Group

In [283]:
groups_dim = create_groups_dim(timetables_combined)

groups_dim.to_excel("data/dim/groups_dim.xlsx", index=True)

### Subject

In [284]:
# subject_dim_tt = create_subjects_dim(timetables_combined, "timetable")
subject_dim_ht = create_subjects_dim(hours_and_teachers_combined, "hours_and_teachers")
subject_dim_ht.to_excel("data/dim/subject_dim.xlsx", index=True)

79


## Teachers

In [285]:
all_teachers = get_all_teachers(hours_and_teachers_combined)


teachers_dim = create_teachers_dim(all_teachers)

teachers_dim.to_excel("data/dim/teachers_dim.xlsx", index=True)

### Rooms

In [287]:
rooms = preprocess_rooms(timetables_combined["room"]).unique()

rooms_dim = create_room_dim(rooms)
rooms_dim.to_excel("data/dim/rooms_dim.xlsx", index=False)